Load necessary packages

In [1]:
import os, sys
import csv

import platform
print('Python: ' + platform.python_version())

import numpy as np
print('numpy: ' + np.__version__)

import pandas as pd
print('pandas: ' + pd.__version__)

import scipy
print('scipy: ' + scipy.__version__)

import sklearn
print('scikit-learn: ' + sklearn.__version__)

import matplotlib as plt
print('matplotlib: ' + plt.__version__)

import flask
print('flask: ' + flask.__version__)

import math

Python: 3.6.1
numpy: 1.13.0
pandas: 0.20.2
scipy: 0.19.0
scikit-learn: 0.18.1
matplotlib: 2.0.2
flask: 0.12.2


Reset if needed

In [2]:
reset -f out

Flushing output cache (0 entries)


Change directory for data analysis

In [3]:
#path_gen = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/11_CFX/05_PhD runs/R67_fluent/peak_R67_hgt_512k/trn_des_v01/' #ścieżka do katalogu z interesującymi nas plikami
path_data = './data/' #ścieżka do katalogu z interesującymi nas plikami
path_post = './post/' #ścieżka do katalogu z interesującymi nas plikami

Load file list

In [4]:
filelist = os.listdir(path_data)            #Tworzy listę z plików w danym katalogu
filelist

FileNotFoundError: [WinError 3] System nie może odnaleźć określonej ścieżki: '/data/'

Define nodes location

In [ ]:
os.chdir(path_data)
coordinates = pd.read_csv('asdp_data-0001.dat', sep='\s+', header=0, usecols=["nodenumber", "x-coordinate", "y-coordinate", "z-coordinate"], skiprows=0, decimal='.')
#coordinates[['nodenumber']] = coordinates[['nodenumber']].astype(int)
coordinates

Add first timestep

In [ ]:
tsnum = str(os.path.basename(str(path_data + filelist[0])))
tsnum = tsnum[10:-4]
tsnum

Add first pressure

In [ ]:
pstatic = pd.read_csv('asdp_data-0001.dat', sep='\s+', header=0, usecols=["pressure"], skiprows=0, decimal='.')
#pstatic[['nodenumber']] = pstatic[['nodenumber']].astype(int) use usecols=["nodenumber", "pressure] if uncommented
acupress = pd.DataFrame(data = pstatic)
acupress.columns = [tsnum]
acupress

In loop add static pressure for other timesteps

In [ ]:
for file in filelist:
    tsnum = str(os.path.basename(str(path_data + file)))
    tsnum = tsnum[10:-4]
    pstatic = pd.read_csv(file, sep='\s+', header=0, usecols=["pressure"], skiprows=0, decimal='.')
    pstatic = pd.DataFrame(data = pstatic)
    pstatic.columns = [tsnum]
    frames = [acupress, pstatic]
    acupress = pd.concat(frames, axis=1)
#acupress = acupress.set_index('nodenumber')
acupress

In [ ]:
acupress.memory_usage(index=True)

Calculate average static pressure for each point

In [ ]:
mean_pressure = []
for index, row in acupress.iterrows():
    row = acupress.iloc[index]
    mean = np.mean(row)
    mean_pressure.append(mean)
mean_pressure = pd.Series(mean_pressure)
mean_pressure

Calculate deviation from average

In [ ]:
deviation = acupress.sub(mean_pressure, axis=0)
deviation

Calculate RMS & DB level

In [ ]:
rms = []
db =[]
for index, row in deviation.iterrows():
    row = deviation.iloc[index]
    point_rms = np.sqrt(np.mean(np.square(row)))
    point_db = 20 * math.log10(point_rms/0.00002)
    rms.append(point_rms)
    db.append(point_db)
    
rms = pd.Series(rms)
db = pd.Series(db)

noise_information = pd.DataFrame({
    'rms': rms,
    'db': db
})
noise_information

Calculate DB level in point for time

In [ ]:
deviation_abs = deviation.abs()
time_db = 20 * np.log10(deviation_abs/0.00002)
time_db

Add scripts for plotting
+ pressure_deviation(time) - for a control point, rms as a horizontal line
+ db(time) - for a control point, ang db as a horizontal line
+ x,y,pressure animated in time
+ x,y, vorticity animated in time

Add scripts fro calculation
+ FFT for control point
+ plot FFT for control point